In [10]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [7]:
import pickle
import pandas as pd

In [11]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [12]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
year = 2021
month = 2

In [22]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month:02d}.parquet')

In [23]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

# Q1

In [24]:
print(f'Answer to question 1 is: {round(y_pred.mean(), 2)}')

Answer to question 1 is: 16.19


# Q2

In [28]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = df.loc[:, ['ride_id']]
df_result['pred'] = pd.Series(y_pred)

output_file = 'output.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [29]:
import os

size = os.path.getsize(output_file)
print(f'Answer to question 2 is: {round(size/(1024*1024), 0)}')

Answer to question 2 is: 19.0


# Q3

In [32]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1496 bytes to starter.py


# Q4

In [38]:
import json

f = open('Pipfile.lock')
data = json.load(f)

In [48]:
data.get('default').get('scikit-learn').get('hashes')[0]

'sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b'

# Q5

In [56]:
from starter import count_mean_pred

In [57]:
df, y_pred = count_mean_pred(2021, 3)

Answer to question 1 is: 16.3


# Q6

`docker build -t duration-prediction:v1 .`

`docker run -it --rm -p 9696:9696 duration-prediction:v1`